# Stroke prediction
## This dataset is used to predict whether a patient is likely to get stroke based on the input parameters
## Attribute Information
1) **id**: unique identifier
2) **gender**: "Male", "Female" or "Other"
3) **age**: age of the patient
4) **hypertension**: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension
5) **heart_disease**: 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease
6) **ever_married**: "No" or "Yes"
7) **work_type**: "children", "Govt_jov", "Never_worked", "Private" or "Self-employed"
8) **Residence_type**: "Rural" or "Urban"
9) **avg_glucose_level**: average glucose level in blood
10) **bmi**: body mass index
11) **smoking_status**: "formerly smoked", "never smoked", "smokes" or "Unknown"*
12) **stroke**: 1 if the patient had a stroke or 0 if not
*Note: "Unknown" in smoking_status means that the information is unavailable for this patient

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
# Original dataset
df_orig = pd.read_csv('W:\Downloads\datasets\healthcare-dataset-stroke-data.csv', index_col='id')
y = df_orig['stroke'].copy()
X_orig = df_orig.drop(columns=['stroke'], inplace=False)
num_feat = len(X_orig.columns) 
num_obj = len(X_orig)
# Кросс-валидация
cv = KFold(n_splits=5, shuffle=True, random_state=42)
#X_orig.head()

# Feature engineering

In [2]:
def ohe(df, feature):
    """
    one-hot enccoder
    """
    categ_list = df[feature].unique()
    df_enc = np.zeros((df.shape[0], len(categ_list)))
    for ii in range(len(categ_list)):
        df_enc[:, ii] = (df[feature]==categ_list[ii]).astype(int) 

    df_enc = pd.DataFrame(data=df_enc, index=df.index, columns=categ_list)
    df_enc = pd.concat([df, df_enc], axis=1)
    return df_enc

# Добавление возрастных групп
age_group = np.zeros(num_obj)
for ii in range(num_obj):
    if X_orig['age'][X_orig['age'].index[ii]] < 25:
        age_group[ii] = 0
    elif X_orig['age'][X_orig['age'].index[ii]] < 45:
        age_group[ii] = 1
    elif X_orig['age'][X_orig['age'].index[ii]] < 61:
        age_group[ii] = 2
    else:
        age_group[ii] = 3
age_group = pd.Series(data=age_group, index=X_orig.index, name='age_group')
X_orig = pd.concat([X_orig, age_group], axis=1)

X_prep = X_orig.copy()
# Закодируем категориальные признаки
# Male - 1, Female - 0
X_prep['gender'] = X_prep['gender'].map({'Male': 1, 'Female': 0, 'Other': 1})
# urban - 1, rural - 0
X_prep['Residence_type'] = X_prep['Residence_type'].map({'Urban': 1, 'Rural': 0})
# ever_married yes - 1, no - 0
X_prep['ever_married'] = X_prep['ever_married'].map({'Yes': 1, 'No': 0})
# work-type (one-hot (dummy) encoder)
X_prep = ohe(X_prep, 'work_type')
X_prep.drop(columns='work_type', inplace=True)
# То же для smoking_status
X_prep = ohe(X_prep, 'smoking_status')
X_prep.drop(columns='smoking_status', inplace=True)
# Заполним пропуски в BMI
X_prep['bmi'].fillna(X_prep['bmi'].median(), inplace=True)
# Age group
X_prep = ohe(X_prep, 'age_group')
X_prep.drop(columns='age_group', inplace=True)
# Отмасштабируем
scaler = StandardScaler()
X_prep = pd.DataFrame(data=scaler.fit_transform(X_prep), index=X_prep.index, columns=X_prep.columns)

# X_prep.drop(columns='Residence_type', inplace=True)
# X_prep.drop(columns='gender', inplace=True)

# 1) Logistic regression

In [ ]:
C_regul = [0.1]
for regul in C_regul:
    clf = LogisticRegression(penalty='l2', C=0.1).fit(X_prep, y)
    print(cross_val_score(clf, X_prep, y, cv=cv, scoring='roc_auc').mean())
# Отложенная выборка
X_train, X_test, y_train, y_test = train_test_split(X_prep, y, test_size=0.3)
clf = LogisticRegression(penalty='l2', C=0.1).fit(X_train, y_train)
#roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
cross_val_score(clf, X_test, y_test, cv=cv, scoring='roc_auc')
# Значимость признаков
# feat_val = pd.DataFrame(data=np.abs(clf.coef_)[0], index=X_prep.columns)
# fig, ax = plt.subplots(figsize=(10, 6))
# feat_val.plot.bar(ax=ax)

# 2) Support Vector Machine (SVM)

Найдем лучшие параметры

In [ ]:
# Find the best value for C regularization parameter
grid_linear = {'C': [0.0001]}
grid_poly = {'C': [0.01, 0.1, 1], 'gamma': [0.001, 0.01, 0.1], 'coef0': [3, 4]}
grid_rbf = {'C': [0.1, 1], 'gamma': [0.001, 0.01, 0.1]}
grid_sigmoid = {'C': [0.001, 0.01, 0.1], 'gamma': [0.0001, 0.01], 'coef0': [1, 10]}
grids = [grid_linear, grid_poly, grid_rbf, grid_sigmoid]
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
params = []
scores = []
for ind, kern in enumerate(kernels):
    svc_clf = SVC(kernel=kern)
    gs = GridSearchCV(estimator=svc_clf, param_grid=grids[ind], cv=cv, scoring='roc_auc')
    gs.fit(X_prep, y)
    params.append(gs.best_params_)
    scores.append(gs.best_score_)
    print(kern, gs.best_params_, gs.best_score_)
params_best = params[np.argmax(scores)]

Обучим на лучших параметрах

In [ ]:
clf_svm = SVC(C=params_best['C'], kernel='rbf', gamma=params_best['gamma'], coef0=params_best['coef0'])
clf_svm.fit(X_prep, y)
# Отложенная выборка
X_train, X_test, y_train, y_test = train_test_split(X_prep, y, test_size=0.3, random_state=42)
clf_svm = SVC(C=params_best['C'], kernel='sigmoid', coef0=10).fit(X_train, y_train)
np.mean(cross_val_score(clf_svm, X_test, y_test, cv=cv, scoring='roc_auc'))


# 3) Decision tree

In [ ]:
X_tree = X_orig.copy()
# Закодируем категориальные признаки
# Male - 1, Female - 0
X_tree['gender'] = X_tree['gender'].map({'Male': 1, 'Female': 0, 'Other': 1})
# urban - 1, rural - 0
X_tree['Residence_type'] = X_tree['Residence_type'].map({'Urban': 1, 'Rural': 0})
# ever_married yes - 1, no - 0
X_tree['ever_married'] = X_tree['ever_married'].map({'Yes': 1, 'No': 0})
# work-type (one-hot (dummy) encoder)
X_tree = ohe(X_tree, 'work_type')
X_tree.drop(columns='work_type', inplace=True)
# То же для smoking_status
X_tree = ohe(X_tree, 'smoking_status')
X_tree.drop(columns='smoking_status', inplace=True)
# Заполним пропуски в BMI
X_tree['bmi'].fillna(X_tree['bmi'].median(), inplace=True)

In [ ]:
clf_tree = DecisionTreeClassifier(criterion='entropy', ccp_alpha=0.003)
clf_tree.fit(X_tree, y)

print(cross_val_score(clf_tree, X_tree, y, cv=cv, scoring='roc_auc').mean())
clf_tree.feature_importances_

# fig, ax = plt.subplots(figsize=(10, 8))
# tree.plot_tree(clf_tree, max_depth=1)

# 4) K Nearest Neighbour

In [13]:
grid = {'n_neighbors': np.arange(1, 10, dtype=int), 'p': np.arange(1, 10, dtype=int)}

clf_knn = KNeighborsClassifier(weights='distance', metric='minkowski')
gs = GridSearchCV(estimator=clf_knn, param_grid=grid, cv=cv, scoring='roc_auc')
gs.fit(X_prep, y)
print(gs.best_params_, gs.best_score_)
# лучшее n = 9, p = 3

{'n_neighbors': 9, 'p': 3} 0.6845260145255814


# 5) Naive Bayes

# Plots

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 6))
# Глюкоза
sns.histplot(X_orig['avg_glucose_level'], ax=ax[0, 0], kde=True)
# bmi glucose
sns.scatterplot(x=X_orig['avg_glucose_level'], y=X_orig['bmi'], ax=ax[1, 0])
# Stroke
ax[0, 1].pie(y.value_counts(), labels=['1', '0'], autopct='%.2f')
# Посмотрим как зависит stroke от gender. Надо смотреть долю, т.к. М и Ж разное кол-во в выборке 
gender_stroke = X_orig['gender'].loc[y==1]
sns.countplot(x=gender_stroke, ax=ax[1, 1])
print(gender_stroke.value_counts()['Female']/X_orig['gender'].value_counts()['Female'], 
        gender_stroke.value_counts()['Male']/X_orig['gender'].value_counts()['Male'])
# Smoking status
fig1, ax1 = plt.subplots(2, 2, figsize=(10, 6))
sns.countplot(y=X_orig['smoking_status'], ax=ax1[0, 0], hue=y)
# Stroke от age
age_stroke = X_orig['age'].loc[y==1]
sns.histplot(x=age_stroke, ax=ax1[1, 0], bins=40, kde=True)
ax1[1, 0].set_ylabel('Number of strokes')
# Распределение возрастов
sns.histplot(x=X_orig['age'], ax=ax1[0, 1], hue=X_orig['gender'], element='step', legend=True)
# BMI
sns.distplot(x=X_orig['bmi'], ax=ax1[1, 1])
sns.distplot(x=X_orig['bmi'].loc[y==1], ax=ax1[1, 1])


In [ ]:
age_stroke = X_orig['age'].loc[y==1]
fig, ax = plt.subplots(figsize=(10, 6))
sns.distplot(X_orig['age'], ax=ax)
sns.distplot(x=age_stroke, ax=ax, bins=20)

In [ ]:
sns.histplot(data=X_orig['age_group'])

## Идеи
- precision-recall curve
- Группы возрастов
- stroke от smoke
- Разные метрики качества
- SVM разные ядра

### Вопросы
- SVM AUC-ROC почему 0.65 всего